# Clustering Consulting Project

## (Henri's Solution + Additional Comments)

A technology start-up in California needs your help!

It's time for you to go to San Francisco to help out a tech startup!

They've been recently hacked and need your help finding out about the hackers!

Luckily their forensic engineers have grabbed valuable data about the hacks, including information like:
- session
- time
- locations
- wpm typing speed
- etc.

The forensic engineer relates to you what she has been able to figure out so far.  She has been able to grab meta-data of each session that the hackers used to connect to their servers.

These are the features of the data.

- ```Session_Connection_Time```:  How long the session lasted in minutes.
- ```Bytes_Transferred```:  Number of MB transferred during the session.
- ```Kali_Trace_Used```:  Indicates if the hacker was using Kali Linux.
- ```Servers_Corrupted```:  The number of server corrupted during the attack.
- ```Pages_Corrupted```:  The number of pages illegally accessed.
- ```Location```:  Location the attack came from (Probably useless because the hackers used VPNs).
- ```WPM_Typing_Speed```:  Their estimated typing speed based on session logs.

Kali Linux is a Debian-derived Linux distribution designed for digital forensics and penetration testing.  It is maintained and funded by Offensive Security Ltd.  It was developed by Mati Aharoni and Devon Kearns of Offensive Security through the rewrite of BackTrack, their previous information security testing Linux distribution based on Knoppix.  The third core developer, Raphael Hertzog, joined them as a Debian expert.

The technology firm has 3 potential hackers that perpetrated the attack.

They are certain of the first two hackers, but they aren't very sure if the third hacker was invovled or not.

They have rerquested your help!

Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just 2 hackers?

It is probably not possible to know for sure, but maybe what you've just learned about clustering can help.

One last key fact, the forensic engineer knows that the hackers trade off attacks.

Meaning they should each have roughly the same amount of attacks.

For example, if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, and in a 3 hacker situation each would have about 33 hacks.

The engineer believes this is the key element to solving this, but does not know how to distinguish this unlabeled data into groups of hackers.

Best of luck with this project, it should be a fun one!

If you get stuck, feel free to go straight to the solution lecture.

Enjoy!

In [1]:
# Create a Spark Session:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("hack").getOrCreate()

In [2]:
data = spark.read.csv("hack_data.csv", inferSchema=True, header=True)

In [3]:
data.printSchema()
print("\n")

print(data.head(1)[0])
print("\n")

data.describe().show()
print("\n")

print(data.count())

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)


+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|              

In [4]:
from pyspark.ml.feature import VectorAssembler

In [5]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [6]:
assembler = VectorAssembler(inputCols=['Session_Connection_Time', 'Bytes Transferred',
                                       'Kali_Trace_Used', 'Servers_Corrupted',
                                       'Pages_Corrupted', 'WPM_Typing_Speed'],
                            outputCol="features")

In [7]:
data_assembled = assembler.transform(dataset=data)

In [8]:
data_assembled.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [9]:
from pyspark.ml.feature import StandardScaler

In [10]:
std_scaler = StandardScaler(inputCol="features", outputCol="scaled_features")

In [11]:
scaler_model = std_scaler.fit(dataset=data_assembled)

In [12]:
data_scaled = scaler_model.transform(dataset=data_assembled)

In [13]:
data_scaled.head(1)[0]

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37, features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), scaled_features=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963]))

### Train the model

In [14]:
from pyspark.ml.clustering import KMeans

In [15]:
kmeans_2 = KMeans(featuresCol="scaled_features", k=2)
kmeans_3 = KMeans(featuresCol="scaled_features", k=3)
kmeans_4 = KMeans(featuresCol="scaled_features", k=4)

In [16]:
kmeans_fitted_2 = kmeans_2.fit(data_scaled)
kmeans_fitted_3 = kmeans_3.fit(data_scaled)
kmeans_fitted_4 = kmeans_4.fit(data_scaled)

In [17]:
print("metric(2) = {:.1f}.".format(kmeans_fitted_2.computeCost(data_scaled)))
print("metric(3) = {:.1f}.".format(kmeans_fitted_3.computeCost(data_scaled)))
print("metric(4) = {:.1f}.".format(kmeans_fitted_4.computeCost(data_scaled)))

metric(2) = 601.8.
metric(3) = 434.1.
metric(4) = 267.1.


### Evaluate the groupings that were made.

In [18]:
df_2 = kmeans_fitted_2.transform(data_scaled).select(["prediction"])
df_3 = kmeans_fitted_3.transform(data_scaled).select(["prediction"])
df_4 = kmeans_fitted_4.transform(data_scaled).select(["prediction"])

In [19]:
from pyspark.sql.functions import count, countDistinct

In [20]:
df_2.groupBy("prediction").count().show()
df_3.groupBy("prediction").count().show()
df_4.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         2|   84|
|         0|  167|
+----------+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   83|
|         3|   84|
|         2|   79|
|         0|   88|
+----------+-----+



***There appears to be 2 hackers involved.  Notice how the counts are identical for the kmeans model where k=2.  The counts are most certainly not equal for the case where k=3.***

***The third suspect had nothing to do with the attacks.  Only the 2 initially suspected hackers were involved.***